In [14]:
# label
label_path0 = './data/회복간호_21상.xlsx'
label_path1 = './data/회복간호_21하.xlsx'
label_path2= './data/회복간호_22상.xlsx'

# meta label
meta_path0 = './data/수술기본_과거력_21상.xlsx'
meta_path1 = './data/수술기본_과거력_21하.xlsx'
meta_path2= './data/수술기본_과거력_22상.xlsx'
# remove nrs-non-existing rows
# select columns
def get_data(path,filter_='simple'):
    df = pd.read_excel(path,header=1)
    df = df.iloc[df[['최대 NRS']].dropna().index]
    if filter_ == 'simple':
        df = df[['등록번호','회복실퇴실일시','최대 NRS']]
    elif filter_ == 'all':
        pass
    return df

### all data stats 

In [4]:
import pandas as pd

In [50]:
def mul2sing(df):
    cols = []
    for c0,c1 in df.columns:
        c0 = '0' if c0.startswith('Unnamed') else c0
        c1 = '0' if c1.startswith('Unnamed') else c1
        cols.append(c0+'_'+c1)
    return cols

m0 = pd.read_excel(meta_path0,header=[0,1],engine='openpyxl')
m1 = pd.read_excel(meta_path1,header=[0,1],engine='openpyxl')
m2 = pd.read_excel(meta_path2,header=[0,1],engine='openpyxl')

m0.columns = mul2sing(m0)
m1.columns = mul2sing(m1)
m2.columns = mul2sing(m2)

m = pd.concat([m0,m1,m2])

m = m[['KEY_0','등록번호_0','연령_0','성별_0','AIDS/HIV_마취종류','AIDS/HIV_수술명','AIDS/HIV_환자 진료과']]

m = m.rename(columns={'AIDS/HIV_마취종류':'anesthesia_type','AIDS/HIV_수술명':'sur_name','AIDS/HIV_환자 진료과':'dept'})

In [69]:
op0 = pd.read_excel(label_path0,header=1,engine='openpyxl')
op1 = pd.read_excel(label_path1,header=1,engine='openpyxl')
op2 = pd.read_excel(label_path2,header=1,engine='openpyxl')

op = pd.concat([op0,op1,op2])

op = op[['KEY','등록번호','회복실퇴실일시','최대 NRS']]

In [ ]:
opm = pd.merge(op,m,how='left',left_on='KEY',right_on='KEY_0')

In [134]:
# only nrs-existing 
opm_nrs = opm.dropna(subset=['최대 NRS'])

In [147]:
# only pdor&pdrec-pair-existing
import pickle
with open('./data/vital_split.pkl','rb') as f:
    allf_s = pickle.load(f)

In [179]:
df = opm_nrs
def get_date(date:str,type_='ymd'):
    
    date = date.split()

    y,m,d = date[0].split('-')
    h,min_   = date[1].split(':')
    
    out = ''
    if type_ == 'ymd':
        ymd = y[-2:]+m+d
        out = ymd
    elif type_ == 'hm':
        hm = h+min_
        out = hm
    return out

df['date'] = df['회복실퇴실일시'].map(lambda x: get_date(x,'ymd') )

/home/yp/anaconda3/envs/torch0/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [190]:
df.columns

Index(['KEY', '등록번호', '회복실퇴실일시', '최대 NRS', 'KEY_0', '등록번호_0', '연령_0', '성별_0',
       'anesthesia_type', 'sur_name', 'dept', 'date'],
      dtype='object')

In [192]:

def has_pair(row):
    hasPair = False
    
    pt_id = str(row['등록번호'])
    date  = str(row['date'])
    
    for i, fnames in enumerate(allf_s): # data: ['00000824', 'ORC14', '210611', '133500', '1']
        if fnames[0].endswith(pt_id): # 등록번호
            if fnames[2].startswith(date): # 회복실퇴실일시(date) 210611
                hasPair=True
                break
                # idx.append([j, sample,allf_n[i],allf[i],sample[-2],sample[-1] ])
    return 

In [ ]:
has = opm_nrs.apply(has_vi)

In [188]:
str(round(12313.111))

'12313'

In [186]:
opm_nrs.iloc[:3].apply(lambda x: print(x.dtype),axis=1)

object
object
object


5     None
6     None
11    None
dtype: object

In [ ]:
# op_list.csv (filter1)
df0_a = get_data(label_path0,filter_='all')
df1_a = get_data(label_path1,filter_='all')
df2_a = get_data(label_path2,filter_='all')

df_a = pd.concat([df0_a,df1_a,df2_a])

df_a = df_a.drop_duplicates(subset=['등록번호','회복실퇴실일시'])

df_a.to_csv('filter0.csv',index=False)